# Developing an integration routine.

In this notebook, I develop a struct and methods that 
describe wiener processes, their refinement, and integration with respect to said wiener process.

I then use that to integrate a single stochastic path.

In [1]:
using Random, Plots
import Statistics
using LinearAlgebra

In [2]:
T = 1.0
N = 25000
Δt = T/N

4.0e-5

In [3]:
struct WienerProcess
    #the two key things to know are 
    path::Vector{Float64}
    Δt::Float64
   
    #This creates a wiener process using a given path, while 
    #enforcing the invariant that a WienerProcess starts at zero
    WienerProcess(p,d) = p[1] != 0.0 ? new(vcat(0.0,p),d) : new(p,d)
end

In [4]:
#Adding methods (not as member functions though)
function get_steps(p::WienerProcess)
    #This pulls the steps out of the WienerProcess
    len = length(p.path)
    if p.path[1] != 0.0
        error("test")
    end 
    steps = p.path[2:len] - p.path[1:len-1]
end

function refine(p::WienerProcess)
    #This refines the resolution of the given WienerProcess, and returns another in it's stead
    len = length(p.path)
    arr = zeros(2*len-1)
    
    for ind in 1:len-1
        arr[2*ind-1] = p.path[ind]
        arr[2*ind] =  (0.5 * (p.path[ind] + p.path[ind+1]) .+ (((√p.Δt)/2)*randn(1)))[1]
    end
    arr[2*len-1] = p.path[len]
    
    return WienerProcess(arr, p.Δt/2)
end

function spread_walk(walk)
    len = length(walk)
    arr = zeros(2*len-1)
    
    for ind in 1:len-1
        arr[2*ind-1] = walk[ind]
        arr[2*ind] =  (0.5 * (walk[ind] + walk[ind+1]) ) 
    end
    arr[2*len-1] = walk[len]
    
    return arr
end

spread_walk (generic function with 1 method)

In [5]:
#Generators
function WienerProcess(size::Int, domain_length::Float64)
    #This function builds a wiener process out of randomly drawn steps
    #it uses the domain length to calculate Δt
    
    #TODO: implement checking for begining with zero
    return WienerProcess(cumsum(randn(size)),domain_length/size)
end

WienerProcess

In [6]:
function integrate(f::Function, wp::WienerProcess)
    #This function integrates across the time domain.
    
    sum = 0
    steps = get_steps(wp)
    
    for i in 1:length(wp.path)-1
        sum += f(wp.path[i]) * (steps[i]) 
    end
    
    return sum
end

integrate (generic function with 1 method)

In [7]:
#Generate a WienerProcess
tp = WienerProcess(N,T)

WienerProcess([0.0, -0.5439574559954317, -1.2142519823462683, -1.5733055705770993, -0.6859904869401457, -0.6392543389335029, -0.9345322012850051, -1.3881939920604034, -1.6028175978216337, -2.1259195470356502  …  -4.040743120978028, -4.066019780054612, -3.2263410618321138, -3.6252673867752203, -5.136073745310993, -5.444987305209882, -5.867429104891739, -6.357154759506193, -7.036421888793718, -5.954543927062083], 4.0e-5)

In [8]:
#test functions
slf(x) = x
unit(x) = 1

unit (generic function with 1 method)

Integrate $f(W_t) = 1$

In [9]:
integrate(unit,tp),tp.path[length(tp.path)]

(-5.954543927062083, -5.954543927062083)

In [10]:
rtp = refine(tp)
integrate(unit,rtp),rtp.path[length(rtp.path)]

(-5.954543927062083, -5.954543927062083)

# Answering the question given
integrate $f(W_t) = W_t$, refining the partition as you go.

In [11]:
function time_refinements(tp, itt = 10)
    iterated_test = tp
    
    refinements = []
    symbolic_integrals = []
    
    for iteration in 1:itt
        iterated_test = refine(iterated_test)
        #append!(refinements, iterated_test)

        integral = @time integrate(slf,iterated_test)

        path_length = length(iterated_test.path)
        las_value = iterated_test.path[path_length]
        symbolic_integral = 0.5 * (las_value^2 - T )

        #Get the values needed of the 
        append!(symbolic_integrals, symbolic_integral)
        
        println("iteration ",iteration, ". Number of discrete points ", path_length)
        println("\t",integral, " " , symbolic_integral)
        println("\tAbsolute Error: ", abs(integral - symbolic_integral))
        println("\tSquared Error: ",(integral - symbolic_integral)^2)
        println("\tPercentage Error: ",(integral - symbolic_integral)/symbolic_integral, "\n")
    end
    #return absolute_error, list_of_Δs
    return refinements, symbolic_integrals
end

time_refinements (generic function with 2 methods)

In [12]:
evaluated_values = [@time time_refinements(tp, 10) for x=1:4]

  0.000394 seconds (6 allocations: 1.145 MiB)
iteration 1. Number of discrete points 50001
	-6269.7534576229455 17.228296689655966
	Absolute Error: 6286.981754312602
	Squared Error: 3.952613957905956e7
	Percentage Error: -364.9218415240878

  0.000796 seconds (6 allocations: 2.289 MiB)
iteration 2. Number of discrete points 100001
	-3126.261381453662 17.228296689655966
	Absolute Error: 3143.489678143318
	Squared Error: 9.88152735659358e6
	Percentage Error: -182.46085116648237

  0.001393 seconds (6 allocations: 4.578 MiB)
iteration 3. Number of discrete points 200001
	-1554.5175919756782 17.228296689655966
	Absolute Error: 1571.7458886653342
	Squared Error: 2.470385138536381e6
	Percentage Error: -91.23048650590196

  0.002526 seconds (6 allocations: 9.156 MiB)
iteration 4. Number of discrete points 400001
	-768.64549320033 17.228296689655966
	Absolute Error: 785.8737898899859
	Squared Error: 617597.6136360497
	Percentage Error: -45.61529233251666

  0.006712 seconds (6 allocations: 18.

4-element Vector{Tuple{Vector{Any}, Vector{Any}}}:
 ([], [17.228296689655966, 17.228296689655966, 17.228296689655966, 17.228296689655966, 17.228296689655966, 17.228296689655966, 17.228296689655966, 17.228296689655966, 17.228296689655966, 17.228296689655966])
 ([], [17.228296689655966, 17.228296689655966, 17.228296689655966, 17.228296689655966, 17.228296689655966, 17.228296689655966, 17.228296689655966, 17.228296689655966, 17.228296689655966, 17.228296689655966])
 ([], [17.228296689655966, 17.228296689655966, 17.228296689655966, 17.228296689655966, 17.228296689655966, 17.228296689655966, 17.228296689655966, 17.228296689655966, 17.228296689655966, 17.228296689655966])
 ([], [17.228296689655966, 17.228296689655966, 17.228296689655966, 17.228296689655966, 17.228296689655966, 17.228296689655966, 17.228296689655966, 17.228296689655966, 17.228296689655966, 17.228296689655966])

In [13]:
#plot(strong_convergence[1][2],Statistics.mean([x[1] for x=strong_convergence])
#    ,yaxis = "mean of abs error"
#    ,xaxis = "delta t")

# Finding the expectation and the variance


In [14]:
observed_paths = [WienerProcess(N,T) for i=1:3000];

In [15]:
evaluated_integrals = [integrate(slf,x) for x=observed_paths];

In [16]:
println("Observed mean: ", Statistics.mean(evaluated_integrals)
    ,"\nObserved var: ", Statistics.var(evaluated_integrals))

Observed mean: 127.68850600942208
Observed var: 2.901842929297139e8


In [17]:
#plot(0.0:Δt:T,[x.path for x=observed_paths[1:10]]
#    , legend=false)